In [1]:
# 먼저 다나와에서 '무선청소기'를 검색한 후, selenium으로 크롬 브라우저를 생성하고 검색결과 페이지 url로 접속

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import pandas as pd

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)  # 브라우저 자동 종료 방지
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])  # 불필요한 에러 메시지 제거

# WebDriver 초기화
driver = webdriver.Chrome(
    service=Service('C:/Users/dbsgu/chromedriver-win64/chromedriver.exe'),
    options=chrome_options
)

# 웹페이지 접속
url = 'https://prod.danawa.com/list/?cate=102207&shortcutKeyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0'
driver.get(url)

In [2]:
# 무선청소기 목록을 보면 각 상품별 상세 스펙과 가격이 나열되어 있다.
# 우측에는 옵션에 따른 가격이 나타난다. 이 중 첫 가격으로 가져와 분석할 것이다.
# 먼저 검색결과의 1페이지에 나온 30개의 상품 정보를 수집한다.

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# 상품정보는 태그명이 li이다. 해당 태그를 select 메서드로 찾고 태그의 개수를 확인한다.

prod_items = soup.select('li.prod_item')
len(prod_items)

31

In [4]:
# 현재 페이지에서 우리 찾는 상품은 30개 이기 때문에 상위 태그 정보를 추가해서 검색한다.

prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
len(prod_items)

30

In [5]:
# 상품정보들 중 상품명, 스펙 목록, 가격을 추출한다.

title = prod_items[0].select('p.prod_name > a')[0].text.strip()
spec_list = prod_items[0].select('div.spec_list')[0].text.strip()
price = prod_items[0].select('p.price_sect > a > strong')[0].text.strip().replace(",","")

print(title)
print(spec_list)
print(price)

삼성전자 비스포크 제트 VS20B956AX
핸디스틱청소기/무선/흡입형/흡입력: 220W/스테이션 기능: 먼지비움, 충전, UVC LED, 스탠드거치/먼지비움시간: 14초/브러쉬: 바닥/물걸레: 별매/솔형/틈새/연장관/먼지봉투: 1.2L/[배터리] 사용시간: 1시간(최대)/분리형(1개)/리튬이온/[청소] LED라이트/싸이클론흡입/디지털인버터모터/워셔블헤파필터/크기(가로x세로x깊이): 250x930x202mm
456820


In [6]:
# 위의 코드를 이용하여 첫 페이지의 30개 상품의 정보를 수집하는 코드를 만든다

prod_data = []

for prod_item in prod_items:
    try: #상품명
        title = prod_items[0].select('p.prod_name > a')[0].text.strip()
    except:
        title = ''
    try: #스펙
        spec_list = prod_items[0].select('div.spec_list')[0].text.strip()
    except:
        spec_list = ''
    try: #가격
        price_list = prod_items[0].select('p.price_sect > a > strong')[0].text.strip().replace(",","")
    except:
        price = 0
    prod_data.append([title, spec_list, price])

print(len(prod_data))
print(prod_data)

30
[['삼성전자 비스포크 제트 VS20B956AX', '핸디스틱청소기/무선/흡입형/흡입력: 220W/스테이션 기능: 먼지비움, 충전, UVC LED, 스탠드거치/먼지비움시간: 14초/브러쉬: 바닥/물걸레: 별매/솔형/틈새/연장관/먼지봉투: 1.2L/[배터리] 사용시간: 1시간(최대)/분리형(1개)/리튬이온/[청소] LED라이트/싸이클론흡입/디지털인버터모터/워셔블헤파필터/크기(가로x세로x깊이): 250x930x202mm', '456820'], ['삼성전자 비스포크 제트 VS20B956AX', '핸디스틱청소기/무선/흡입형/흡입력: 220W/스테이션 기능: 먼지비움, 충전, UVC LED, 스탠드거치/먼지비움시간: 14초/브러쉬: 바닥/물걸레: 별매/솔형/틈새/연장관/먼지봉투: 1.2L/[배터리] 사용시간: 1시간(최대)/분리형(1개)/리튬이온/[청소] LED라이트/싸이클론흡입/디지털인버터모터/워셔블헤파필터/크기(가로x세로x깊이): 250x930x202mm', '456820'], ['삼성전자 비스포크 제트 VS20B956AX', '핸디스틱청소기/무선/흡입형/흡입력: 220W/스테이션 기능: 먼지비움, 충전, UVC LED, 스탠드거치/먼지비움시간: 14초/브러쉬: 바닥/물걸레: 별매/솔형/틈새/연장관/먼지봉투: 1.2L/[배터리] 사용시간: 1시간(최대)/분리형(1개)/리튬이온/[청소] LED라이트/싸이클론흡입/디지털인버터모터/워셔블헤파필터/크기(가로x세로x깊이): 250x930x202mm', '456820'], ['삼성전자 비스포크 제트 VS20B956AX', '핸디스틱청소기/무선/흡입형/흡입력: 220W/스테이션 기능: 먼지비움, 충전, UVC LED, 스탠드거치/먼지비움시간: 14초/브러쉬: 바닥/물걸레: 별매/솔형/틈새/연장관/먼지봉투: 1.2L/[배터리] 사용시간: 1시간(최대)/분리형(1개)/리튬이온/[청소] LED라이트/싸이클론흡입/디지털인버터모터/워셔블헤파필터/크기(가로x세로x깊이): 250x930x202mm', '456820'], ['삼성전자 비

In [7]:
def get_prod_items(prod_items):
    prod_data = []

    for prod_item in prod_items:
    #상품명
        try:
            title = prod_item.select('p.prod_name > a')[0].text.strip()
        except:
            title = ''
    #스펙
        try:
            spec_list = prod_item.select('div.spec_list')[0].text.strip()
        except:
            spec_list = ''
    #가격
        try:
            price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",", "")
        except:
            price = 0
        
        prod_data.append([title, spec_list, price])

    return prod_data

In [8]:
# 여러 페이지를 반복적으로 처리할 것이어서 url을 만드는 함수를 별도로 만들었다.

def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumneType=allvs&page={}&limit=30&sork=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)

http://search.danawa.com/dsearch.php?query=무선청소기&volumneType=allvs&page=1&limit=30&sork=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods


In [9]:
# 크롤링 과정이 길 것으로 예상되기 때문에 tpdm 모듈을 이용하여 진행 단계를 파악한다.

import time
from tqdm.notebook import tqdm

total_page = 10
for page in tqdm(range(1, total_page + 1)):
    # 페이지가 로딩 완료되기 위한 시간을 5초로 한다
    time.sleep(5)

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
# 이전에는 한 페이지의 상품 정보를 수집했다면 이번에는 상품 검색 결과에서 전체 페이지의 상품 정보를 수집한다.

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)  # 브라우저 자동 종료 방지
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])  # 불필요한 에러 메시지 제거

# WebDriver 초기화
browser = webdriver.Chrome(
    service=Service('C:/Users/dbsgu/chromedriver-win64/chromedriver.exe'),
    options=chrome_options
)
driver.implicitly_wait(3)

keyword = '무선청소기'
total_page = 10
prod_data_total = []

# 진행 정도를 보여주는 tqdm
for page in tqdm(range(1, total_page + 1)):
    url = get_search_page_url(keyword, page)
    driver.get(url)
    time.sleep(5)
    # 현재 페이지의 html 정보
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # 상품 정보 추출
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    # 추출 데이터 저장
    prod_data_total = prod_data_total + prod_item_list

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
data.to_excel('C:/Users/dbsgu/danawa_crawling_result.xlsx', index = False)